In [1]:
import csv
from astropy.io import fits
import numpy as np
import sys
import math
import re

In [2]:
x1 = 5.5957
y1 = -0.907
x2 = 5.61658
y2 = 0.258

m = (y2-y1)/(x2-x1)
b = -0.616-(m*5.6632)

def slope(x):
    return m*x+b

def cutter(file,output_name):
    
    '''Used to cut out the far left part of the field of view.'''
    
    RA = []
    Dec = []
    RA2000 = []
    Dec2000 = []
    zmag = []
    ymag = []
    jmag = []
    hmag = []
    kmag = []
    zobsmag = []
    yobsmag = []
    jobsmag = []
    hobsmag = []
    kobsmag = []
    zaper = []
    yaper = []
    japer = []
    haper = []
    kaper = []
    zapererr = []
    yapererr = []
    japererr = []
    hapererr = []
    kapererr = []
    
    bad_points = []
    
    hdul = fits.open(file)
    allpoints = list(range(len(hdul[1].data)))
    
    for i in range(len(hdul[1].data)):
        x = hdul[1].data[i]['RA']
        chi = int(x[0:2]) + int(x[3:5])/60 + float(x[6::])/3600
        
        y = hdul[1].data[i]['Dec']
        if y[0] == '+':
            ups = int(y[1:3]) + int(y[4:6])/60 + float(y[7::])/3600
        else:
            ups = -1*(int(y[1:3]) + int(y[4:6])/60 + float(y[7::])/3600)
    
        if (slope(chi) - (ups)) > 0:
            bad_points.append(i)
    
    output_fits = output_name + '.fits'
    output_reg = output_name + '.reg'
    
    for x in bad_points:
        allpoints.remove(x)
    
    for j in allpoints:
        RA.append(hdul[1].data['RA'][j])
        Dec.append(hdul[1].data['Dec'][j])
        RA2000.append(hdul[1].data['RA2000'][j])
        Dec2000.append(hdul[1].data['Dec2000'][j])
        zmag.append(hdul[1].data['Mag_Z'][j])
        ymag.append(hdul[1].data['Mag_Y'][j])
        jmag.append(hdul[1].data['Mag_J'][j])
        hmag.append(hdul[1].data['Mag_H'][j])
        kmag.append(hdul[1].data['Mag_K'][j])
        zobsmag.append(hdul[1].data['Obs_Mag_Z'][j])
        yobsmag.append(hdul[1].data['Obs_Mag_Y'][j])
        jobsmag.append(hdul[1].data['Obs_Mag_J'][j])
        hobsmag.append(hdul[1].data['Obs_Mag_H'][j])
        kobsmag.append(hdul[1].data['Obs_Mag_K'][j])
        zaper.append(hdul[1].data['Aper_Z'][j])
        yaper.append(hdul[1].data['Aper_Y'][j])
        japer.append(hdul[1].data['Aper_J'][j])
        haper.append(hdul[1].data['Aper_H'][j])
        kaper.append(hdul[1].data['Aper_K'][j])
        zapererr.append(hdul[1].data['Aper_Z_err'][j])
        yapererr.append(hdul[1].data['Aper_Y_err'][j])
        japererr.append(hdul[1].data['Aper_J_err'][j])
        hapererr.append(hdul[1].data['Aper_H_err'][j])
        kapererr.append(hdul[1].data['Aper_K_err'][j])
    
    col1 = fits.Column(name='RA',format='11A',unit='dms',array=RA)
    col2 = fits.Column(name='Dec',format='11A',unit='dms',array=Dec)
    col3 = fits.Column(name='RA2000',format='D',unit='radians',array=RA2000)
    col4 = fits.Column(name='Dec2000',format='D',unit='radians',array=Dec2000)
    col5 = fits.Column(name='Mag_Z',format='D',unit='mag',array=zmag)
    col6 = fits.Column(name='Obs_Mag_Z',format='D',unit='mag',array=zobsmag)
    col7 = fits.Column(name='Aper_Z',format='E',unit='ADU',array=zaper)
    col8 = fits.Column(name='Aper_Z_err',format='E',unit='ADU',array=zapererr)
    col9 = fits.Column(name='Mag_Y',format='D',unit='mag',array=ymag)
    col10 = fits.Column(name='Obs_Mag_Y',format='D',unit='mag',array=yobsmag)
    col11 = fits.Column(name='Aper_Y',format='E',unit='ADU',array=yaper)
    col12 = fits.Column(name='Aper_Y_err',format='E',unit='ADU',array=yapererr)
    col13 = fits.Column(name='Mag_J',format='D',unit='mag',array=jmag)
    col14 = fits.Column(name='Obs_Mag_J',format='D',unit='mag',array=jobsmag)
    col15 = fits.Column(name='Aper_J',format='E',unit='ADU',array=japer)
    col16 = fits.Column(name='Aper_J_err',format='E',unit='ADU',array=japererr)
    col17 = fits.Column(name='Mag_H',format='D',unit='mag',array=hmag)
    col18 = fits.Column(name='Obs_Mag_H',format='D',unit='mag',array=hobsmag)
    col19 = fits.Column(name='Aper_H',format='E',unit='ADU',array=haper)
    col20 = fits.Column(name='Aper_H_err',format='E',unit='ADU',array=hapererr)
    col21 = fits.Column(name='Mag_K',format='D',unit='mag',array=kmag)
    col22 = fits.Column(name='Obs_Mag_K',format='D',unit='mag',array=kobsmag)
    col23 = fits.Column(name='Aper_K',format='E',unit='ADU',array=kaper)
    col24 = fits.Column(name='Aper_K_err',format='E',unit='ADU',array=kapererr)
    
    cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24])
    
    '''Here we make the fits file'''

    data = fits.BinTableHDU.from_columns(cols)
    data.writeto(output_fits)

    '''Now we make the reg file'''

    pos_data = []

    for v in range(len(RA)):

        pos_data.append([RA[v],Dec[v]])

    with open(output_reg, 'w', newline='') as csvfile:
        reg_file = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_ALL)
        for w in range(len(pos_data)):
            reg_file.writerow([pos_data[w][0] , pos_data[w][1]])

In [3]:
cutter('final_3filt_wmissing.fits','final_3filt_wmissing_cut')